# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのスターターノートブックであり、特に「Rigging」というフレームワークを使用して、LLM（大規模言語モデル）が「20の質問」ゲームをプレイできるようにするために作成されています。

### 問題の内容
- コンペティションでは、参加者が言語モデルを使用して、限られた質問を通じて指定された単語を当てるゲームを行います。目指すべきは、最小限の質問で答えに到達することであり、そのためには効果的な質問を生成し、的確な回答を引き出すことが求められます。
- Notebook内では、`llama3`モデルを`phi3`モデルに切り替えて性能を改善し、LLMと非LLMエージェントを組み合わせた戦略を取っています。Riggingフレームワークを使用することで、異なるモデル間のインタラクションを容易にし、期待される出力を確認しながらLLMにクエリを送信できます。

### 使用されている手法とライブラリ
- **Rigging**: Pydantic XMLに基づいた軽量のLLMインタラクションフレームワークで、LLMの生産パイプラインを管理します。質問の生成や回答の取得を柔軟に行うことができます。
- **vLLM**: モデルをローカルサービスとしてホストするために使用され、データの取得やモデルのトレーニング環境を整える役割を果たします。
- 追加のライブラリとして、`huggingface_hub`や`pandas`が使用され、モデルの重みのダウンロードやデータ管理を効率化しています。

### ノートブックのワークフロー
1. **初期設定**: Hugging FaceとKaggleのトークンの取得、依存パッケージのインストールが行われる。
2. **モデルの準備**: `snapshot_download`を通じてHugging Faceからモデルの重みをダウンロードします。
3. **vLLMサーバーの起動**: ローカル環境でモデルをホストし、LLMが質問と回答を行えるようにします。
4. **質問者と回答者のエージェント作成**: Pythonクラスを利用して、質問を生成し、回答を処理するロジックが整備されています。
5. **キーワードデータフレームの作成**: 公開テーマについてのキーワードが格納され、どの単語に対して質問するかを決定するためのベースになります。
6. **最終提出物の作成**: 実行環境の準備が整い、`main.py`を含むZIPファイルが作成されます。

このノートブックは、Kaggleのコンペティション用にLLMを効果的に活用できる基盤を提供しており、Riggingを利用することで、競技者が簡単にLLMのインタラクションを構築し、テストし、調整できるように設計されています。

---


# 用語概説 
以下は、Jupyter Notebookに関連する専門用語の解説です。特に、初心者がつまずきやすいマイナーな用語や実務を経験していないと馴染みがない用語に焦点を当てました。

1. **Rigging**:
   - 軽量なLLMインタラクションフレームワークであり、Pydantic XMLに基づいて構築されています。特に、バックエンドのLLMモデルを簡単に入れ替えたり、LLMクエリパイプラインを設計するために使用されます。LLM（大規模言語モデル）とのインタラクションを効率化することを目的としています。

2. **Pydantic**:
   - Pythonのデータバリデーションと設定管理を行うためのライブラリです。特に、データモデルの定義を簡単に行える機能を提供し、型ヒントを用いたバリデーションが可能です。

3. **vLLM**:
   - モデルをローカルでホストするためのライブラリであり、特にLLMを効率的にサーバーとして運用するためのツールです。特に非同期的な処理をサポートしており、高速な応答が求められる環境に適しています。

4. **Activation-aware Weight Quantization**:
   - モデルの軽量化手法の一つで、モデルのパラメータを量子化（サイズを小さくする）する際に、活性化のビット幅も考慮する技術です。この手法は、モデルの推論時のパフォーマンスを改善するために使用されることがあります。

5. **XML (eXtensible Markup Language)**:
   - データの構造を記述するためのマークアップ言語で、特にデータの交換や保存に用いられます。このノートブックでは、LLMの入力や出力フォーマットとして使用されています。

6. **ChatPipeline**:
   - Riggingにおいて、LLMとの対話の流れを定義するためのモデルであり、特定のチャットの操作や応答管理を担います。LLMとのインタラクションを効率化するための一連のステップを含んでいます。

7. **Subprocess**:
   - Pythonで別のプログラムやスクリプトを実行するためのモジュールであり、特に外部コマンドを効率的に呼び出す際に使用されます。ここでは、vLLMサーバーをバックグラウンドで起動するために用いられています。

8. **Parsing**:
   - 入力されたデータを特定の形式や構造に従って解釈したり変換したりするプロセスです。LLMの出力を適切な形式で抽出するために使用されます。

9. **Field Validator**:
   - Pydanticでモデルのフィールド値が要件を満たしているかを確認するためのメソッドで、データの整合性を保証する役割を担います。特にそのモデル内で定義されます。

10. **Observation**:
    - ゲームの進行状況や状態を保持するためのデータ構造で、質問や回答、推測をトラックするのに使用されます。ゲームのロジックを管理するために重要な役割を果たします。

これらの用語は、Notebookでの具体的な実例やコードに関連しているため、より理解を深めるための助けとなるでしょう。

---


# LLM 20 Questions スターター with Rigging [Phi3]

## このノートブックは、以前のノートブックの改訂版です。`llama3`モデルを@bhanupmのリクエストに応じて`phi3`モデルに置き換えました。元のノートブックは[こちら](https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/)で見つけることができます。

初期のテストでは、llama3バージョンがより良いパフォーマンスを発揮しましたが、より良いプロンプトによって改善される可能性があります。

このスターターノートブックでは、コンペティションのためのベースライン提出を作成する方法を示します。このセットアップは、vLLMを使用した`phi3`量子化モデルを利用しています。

## 更新 **2024年6月10日**
- rigging 2.0に対応するようにコードを更新しました
- Known keywordsを活用する非LLM質問エージェントを含めました **これはプライベートリーダーボードではうまく機能しません**。回答エージェントはriggingを介してLLMを使用します。

## Riggingとは？

Riggingは、Pydantic XMLに基づいて構築された軽量なLLMインタラクションフレームワークです。目的は、LLMを生産パイプラインでできるだけ簡単かつ効果的に活用できるようにすることです。Riggingは20の質問タスクに完全に適しており、以下のことが可能です：
1. 異なるバックエンドLLMモデルを簡単に入れ替えることができます。
2. 期待される出力をチェックし、成功するまで繰り返すLLMクエリパイプラインを設計できます。
3. 現代のPythonで、型ヒント、非同期サポート、pydanticバリデーション、シリアル化などをサポートしています。

リポジトリをこちらでスターしておいてください: https://github.com/dreadnode/rigging
ドキュメントをこちらでお読みください: https://rigging.dreadnode.io/

Riggingは[dreadnode](https://www.dreadnode.io/)によって構築され、維持されています。私たちは日常的にそれを使用しています。

例として、以下のようなriggingパイプラインが考えられます：
```{python}
chat = rg.get_generator('gpt-4o') \
    .chat(f"Provide me the names of all the countries in South America that start with the letter A {Answer.xml_tags()} tags.") \
    .until_parsed_as(Answer) \
    .run() 
```

生成器は、大多数の主要なLLM APIとシームレスに作成できます。APIキーが環境変数として保存されている限り、問題ありません。
```
export OPENAI_API_KEY=...
export TOGETHER_API_KEY=...
export TOGETHERAI_API_KEY=...
export MISTRAL_API_KEY=...
export ANTHROPIC_API_KEY=...
```

このコンペティションではモデルをローカルで実行する必要があるため、幸運にもriggingはトランスフォーマーをバックエンドに使用してモデルを実行するサポートがあります。

# セットアップ

以下は、このノートブックのいくつかのセットアップです。ここでは以下のことを行います：
- Hugging FaceとKaggleのための秘密トークンをロードします（オプション）
- 必要なパッケージをインストールします
- vLLMサーバーをテストするためのヘルパーユーティリティスクリプトを作成します

このノートブックは、Kaggleの秘密を使用していくつかの隠されたトークンを利用します。これはオプションであり、コードを実行するためには必要ありません。


In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass

## パッケージのインストール
私たちは以下をインストールします：
- [rigging](https://github.com/dreadnode/rigging) コンペティションのためのLLMパイプラインを作成するために使用します。
- [vLLM](https://github.com/vllm-project/vllm) モデルをローカルで独立したサービスとしてホストするために使用します。

また、[uv](https://github.com/astral-sh/uv)を使用しており、これによりこれらのパッケージをより迅速にインストールできます。

**注意:** これらのパッケージは`/kaggle/tmp/lib`ディレクトリにインストールされています。これはコンペティションのセットアップ目的のために行い、このパス内のファイルを後で提出用のZIPに含める必要があります。また、vllmの依存パッケージも`/kaggle/tmp/srvlib`にインストールします。


In [ ]:
# 依存関係（速度向上のためのuv）
!pip install uv==0.1.45

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4

# LLMをローカルにダウンロードする

このコンペティションでは、モデルの重みを含むコードを提出する必要があるため、まず`snapshot_download`を使用してHugging Faceからモデルの重みをダウンロードします。

`solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ`をダウンロードします。これは、コンペティション要件に十分に小さいActivation-aware Weight Quantizationバージョンのモデルです。

**注意:** 通常の状況でriggingを使用する場合、このステップは必要ありませんが、コンペティション用に提出物に含めるために重みを別々にダウンロードしています。


In [ ]:
# モデルをダウンロードする

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="rhysjones/Phi-3-mini-mango-1-llamafied",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)

モデルの重みが`/kaggle/tmp/model/`に保存されているのが確認できます。


In [ ]:
!ls -l /kaggle/tmp/model

# ヘルパーユーティリティファイル

これらは、vLLMサーバーを起動するために使用するヘルパー関数です。


In [ ]:
%%writefile util.py

# vLLMサーバーを起動するためのヘルパー

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                return True
    except socket.error:
        pass
    
    return False

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60, debug: bool = False
) -> subprocess.Popen:
    
    if check_port(port):
        raise ValueError(f"ポート {port} はすでに開いています")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},
        stdout=subprocess.DEVNULL if not debug else None,
        stderr=subprocess.DEVNULL if not debug else None,
    )
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen
        time.sleep(1)
    
    popen.terminate()
    raise Exception(f"プロセスが {port} を {timeout} 秒以内に開きませんでした。")

# テストのためにvLLMサーバーを起動する

私たちのモデルは、vLLMサーバーを使用してホストされます。以下では、Kaggle環境での動作を理解するためにノートブックを起動します。


In [ ]:
# vLLMのパスと設定。

import importlib
from pathlib import Path
import util

util = importlib.reload(util)

g_srvlib_path = Path("/kaggle/tmp/srvlib")
assert g_srvlib_path.exists()

g_model_path = Path("/kaggle/tmp/model")
assert g_model_path.exists()

g_vllm_port = 9999
g_vllm_model_name = "custom"

In [ ]:
# サブプロセスを使用してvLLMサーバーを実行する
vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),
    "--port", str(g_vllm_port),
    "--served-model-name", g_vllm_model_name,
    "--dtype=half"
],
    g_vllm_port,
    {"PYTHONPATH": str(g_srvlib_path)},
    debug=False
)

print("vLLMが起動しました")

llama3モデルが最初のTesla T4 GPUにロードされているのが確認できます。


In [ ]:
!nvidia-smi

## モデルの検証

最初のrigging生成器を作成します。riggingにおいて、生成器は強力なLLMパイプラインを作成する基本です。


In [ ]:
# Riggingとの接続

import sys
import logging

sys.path.insert(0, "/kaggle/tmp/lib")

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

import rigging as rg

generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

answer = await generator.chat("Say Hello!").run()

print()
print('[Rigging Chat]')
print(type(answer), answer)

print()
print('[LLMのレスポンスのみ]')
print(type(answer.last), answer.last)

print()
answer_string = answer.last.content
print('[LLMレスポンスを文字列として]')
print(answer.last.content)

## 結果をpandasデータフレームに変換する

`to_df()`メソッドを使用することで、チャット履歴を簡単にpandasデータフレームに変換できます。


In [ ]:
answer.to_df()

## モデルパラメーターの変更

データベース接続文字列と同様に、Rigging生成器は使用するプロバイダー、モデル、APIキー、生成パラメータなどを定義する文字列として表現できます。それらは以下の形式にフォーマットされます：

```
<provider>!<model>,<**kwargs>
```

例えば、以下のように追加のパラメータを使用してモデルをロードします：
- temperature=0.9
- max_tokens=512

これらについての詳細は、ドキュメントのこちらをお読みください: https://rigging.dreadnode.io/topics/generators/#overload-generation-params


In [ ]:
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=0.9,max_tokens=512," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

また、`rg.GenerateParams`クラスを使用してこれらのパラメータを設定することもできます。このクラスを使用すると、さまざまなモデルパラメータを設定できます：

```
rg.GenerateParams(
    *,
    temperature: float | None = None,
    max_tokens: int | None = None,
    top_k: int | None = None,
    top_p: float | None = None,
    stop: list[str] | None = None,
    presence_penalty: float | None = None,
    frequency_penalty: float | None = None,
    api_base: str | None = None,
    timeout: int | None = None,
    seed: int | None = None,
    extra: dict[str, typing.Any] = None,
)
```

https://rigging.dreadnode.io/api/generator/#rigging.generator.GenerateParams


In [ ]:
rg_params = rg.GenerateParams(
    temperature = 0.9,
    max_tokens = 512,
)
base_chat = generator.chat(params=rg_params)
answer = await base_chat.fork('How is it going?').run()
print(answer.last.content)

また、パラメータはチェーン内でparamsを使用して設定することもできます。


In [ ]:
base_chat = generator.chat() # パラメータは設定されていません
answer = await base_chat.fork('How is it going?') \
    .with_(temperature = 0.9, max_tokens = 512) \
    .run()
print(answer.last.content)

# パースされた出力の例

次に、以下の手順を実行するパイプラインを作成します：
1. `Answer`というriggingモデルを作成します。これにより、モデルの結果からパースする期待出力が説明されます。
    - ここで、出力が`yes`または`no`であることを確実にするために、いくつかのバリデーターを追加します。
    - これは完全にカスタマイズ可能です。
    - ここで`validate_content`は、私たちのレスポンスが期待される出力（小文字で始まり、"yes"または"no"で始まる）に準拠していることを確認します。
2. 私たちは、プロンプトに`Answer.xml_example()`を使用して、LLMに出力がどのように見えるべきかを知らせます。
3. 後で`.until_parsed_as(Answer)`を使用して、LLMの出力がここで定義されたように抽出されることを保証します。

**注意** `until_parsed_as()`は、デフォルトで5の`max_rounds`パラメータを取ることができます。


In [ ]:
import typing as t
from pydantic import field_validator

class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"

In [ ]:
# このxmlの例がプロンプト内でどのように表示されるかを見てみましょう
Answer.xml_example()

In [ ]:
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
#     "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

keyword='Tom Hanks'
category='有名な人'
last_question='それは有名な人ですか？'

prompt = f"""\
            このゲームの秘密の言葉は "{keyword}" です [{category}]

            現在、上記の言葉についての質問に答えています。

            次の質問は "{last_question}" です。

            上記のはい/いいえの質問に答え、以下のフォーマットにして下さい：
            {Answer.xml_example()}

            - あなたのレスポンスは、上記のキーワードに対して正確であるべきです
            - 常に "yes" または "no" で回答してください

            答えは何ですか？
"""

chat = await (
    generator
    .chat(prompt)
    .until_parsed_as(Answer, max_rounds=50)
    .run()
)

print('=== フルチャット ===')
print(chat)

print()
print('=== LLMのレスポンスのみ ===')
print(chat.last)

print()
print('=== パースされた回答 ===')
print(chat.last.parse(Answer).content)

# Riggingで質問者チャットパイプラインの例を作成する

次に、キーワードが何であるかを特定するために役立つ質問者パイプラインを作成します。

まず、出力をパースするために使用する`Question`オブジェクトを作成します。


In [ ]:
from pydantic import StringConstraints  # noqa

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**質問**").to_pretty_xml()

In [ ]:
base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")


question_chat = await (base.fork(
    f"""\
    あなたは現在次の質問をしています。

    質問を以下のフォーマットにして下さい：
    {Question.xml_example()}

    - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
    - 質問は一般的なものから始めてください
    - 残りの探索空間を二分するようにして下さい
    - 以前の質問と回答に注意してください

    あなたの次の質問は何ですか？
    """
)
.until_parsed_as(Question, attempt_recovery=True)
.run()
)

In [ ]:
# 会話のデータフレーム表現
question_chat.to_df()

私たちはLLMのレスポンスが質問を含んでいることを自信を持って確認し、以下のように質問を抽出することができます：


In [ ]:
question = question_chat.last.parse(Question).content
print(question)

# キーワードデータフレームを作成する
** これは、公開されたセットの中での可能なキーワードを知っているためのみ機能します。最終的なリーダーボードでは機能しません**


In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
!head keywords_local.py

In [ ]:
import sys
import json
import pandas as pd
sys.path.append('./')
from keywords_local import KEYWORDS_JSON

def capitalize_first_word(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()

def create_keyword_df(KEYWORDS_JSON):
    keywords_dict = json.loads(KEYWORDS_JSON)

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    for d in keywords_dict:
        words = [w['keyword'] for w in d['words']]
        cat_word = [(d['category'], w['keyword']) for w in d['words']]
        category_words_dict[d['category']] = words
        all_words += words
        all_cat_words += cat_word

    keyword_df = pd.DataFrame(all_cat_words, columns=['category','keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]
    keyword_df.to_parquet('keywords.parquet')
    
create_keyword_df(KEYWORDS_JSON)

In [ ]:
keywords_df = pd.read_parquet('keywords.parquet')
keywords_df.sample(10)

In [ ]:
keywords_df['category'].value_counts()

# 最終提出用の`main.py`スクリプトを作成する。

私たちの最終提出物は、`main`ファイルを含むZIP圧縮ディレクトリになります。以下にそのファイルがあります。


In [ ]:
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# ロギングのノイズを無効にする

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# 固定されたインポート

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
)

# タイプ

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="質問").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="物/場所").to_pretty_xml()


# 関数

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # キーワードバグが修正されるまで、最初の質問をオーバーライド
        return "私たちは20の質問をプレイしていますか？"
    
    
    full_question = f"""\
                あなたは現在次の質問をしています。

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                上記の履歴に基づき、次に最も有用なはい/いいえの質問をし、以下のフォーマットにしてください：
                {Question.xml_example()}

                - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
                - 質問は一般的なものから始めてください
                - 残りの探索空間を二分してください
                - 以前の質問と回答に注意してください

                あなたの次の質問は何ですか？
                """
    
    print(' ======質問中 ======')
    print(full_question)
    
    
    try:
        chat = await (
             base.fork(full_question)
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return '人ですか？'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("回答者にキーワードが提供されていません", file=sys.stderr)
        return "yes" # キーワードバグが修正されるまでオーバーライド
    
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            # 5回ループし、最も頻繁なレスポンスを取得
            chat = await (
                base.fork(
#                     f"""\
#                         20の質問ゲーム。次のキーワードに対してはい/いいえで回答してください：[ {observation.keyword} ]

#                             質問: {last_question}

#                             ルール:
#                             1.  [{observation.keyword}] のみを考慮してください
#                             2. 各文字の質問のために文字を確認してください
#                             3. はいまたはいいえのみで答えてください

#                             フォーマット:
#                             <answer>yes</answer>
#                             または
#                             <answer>no</answer>

#                             あなたの回答は：
#                         """
                    f"""
                    キーワード: [ {observation.keyword} ]

                    Q: {last_question}

                    フォーマット: <answer>yes</answer> または <answer>no</answer>ではいまたはいいえで答えてください
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'レスポンスは {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% エラーが発生したため、yesを回答します %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:

        chat = await (
            base.fork(
                f"""\
                あなたは現在キーワードの情報に基づく推測を行っています。

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                上記の歴史に基づき、キーワードの次に最適な推測を1つ生成し、以下のフォーマットにしてください：
                {Guess.xml_example()}

                - 上記の歴史に基づいて重複した推測を避けてください
                - 推測は特定の人物、場所、または物である必要があります

                あなたの推測は何ですか？
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'フランス'
    
# vLLMとGenerator

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name,
        "--dtype=half"
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLMが起動しました")
except ValueError:
    print('vLLMはすでに実行中です')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")

# エントリーポイント
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# シンプルな質問者
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "私たちは20の質問をプレイしていますか？",
            "キーワードは場所ではないものですか？",
            "キーワードは場所ですか？",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # 以前の回答に基づいてkeyword_dfをフィルタリング
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "キーワードは場所ではないものですか？":
                        self.found_category = 'things'
                    if obs.questions[i] == "キーワードは場所ですか？":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('キーワードは次の文字で始まりますか'):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('サンプル文字', n_letters, sample_letters)
        return sample_letters # ', '.join(sample_letters)
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"キーワードは次のいずれかのものでしょうか？ {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # 推測された単語を取り除く
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # dfをリセット
#         print(f'ラウンド {self.round}')
#         print(f"{response=}")
#         print(f'keyword_dfサイズ {self.keyword_df.shape}')
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
                return question_agent(obs)

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # 彼らのフレームワークで実行するときの非同期ゲート
    import asyncio
    return asyncio.run(observe(obs))

# エージェントを自己評価する


In [ ]:
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

format_first_letter_question(['a','b','c'])

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

In [ ]:
%load_ext autoreload
%autoreload 2
from main import Observation, agent_fn, observe

In [ ]:
# vllmが実行中か確認
!ps -aef | grep vllm

In [ ]:
import pandas as pd

keyword_df = pd.read_parquet('keywords.parquet')
sample = keyword_df.sample(1)

obs = Observation(step = 0,
    role = 'guesser',
    turnType= "ask",
    keyword= sample['keyword'].values[0],
    category= sample['category'].values[0],
    questions = [],
    answers= [],
    guesses= [],
)

question_agent = None

for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)
    print(f'[{i} 質問]: {question}')
    obs.questions.append(question)
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)
    obs.answers.append(answer)
    
    if obs.questions[-1].startswith('私たちは20の質問をプレイしていますか？'):
        gt_answer = answer # 何でも
    elif obs.questions[-1].startswith('キーワードは場所ではないものですか？'):
        if sample['category'].values[0] == 'things':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは場所ですか？'):
        if sample['category'].values[0] == 'place':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは次の文字で始まりますか'):
        letters_guess = extract_letters_from_question(obs.questions[-1])
        gt_answer = obs.keyword[0] in letters_guess
        gt_answer = 'yes' if gt_answer else 'no'
    elif obs.questions[-1].startswith('キーワードは次のいずれかのものでしょうか？'):
        possible_kw = obs.questions[-1].replace('キーワードは次のいずれかのものでしょうか？ ','').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]
        print(possible_kw)
        gt_answer = obs.keyword in possible_kw
        gt_answer = 'yes' if gt_answer else 'no'

    print(f'[{i} 回答]: {answer} [真の回答]: {gt_answer}')
    if answer != gt_answer:
        break

    obs.role = 'guesser'
    obs.turnType = 'guess'
    guess = await observe(obs)
    print(f'[{i} 推測]: {guess} - [キーワード]: {obs.keyword}')
    obs.guesses.append(guess)
    if guess == obs.keyword:
        print('正解です！')
        break
        
    obs.step += 1

In [ ]:
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたはキーワード[{obs.keyword}]に対してはい/いいえの質問に正確に答えます。カテゴリは[{obs.category}]です。

# [質問] "{last_question}" [/質問]

# キーワードは[{obs.keyword}]です - 質問にはこの特定のキーワードのために正確に答えてください。

# 上記のはい/いいえの質問に答え、以下のフォーマットにしてください：
# {Answer.xml_example()}
# """).run()

In [ ]:
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたは20の質問ゲームをプレイしています。あなたの任務は特定のキーワードについてはっきりとしたはい/いいえの質問に答えることです。

# キーワード: [{obs.keyword}]
# カテゴリ: [{obs.category}]

# 質問: {last_question}

# 指示:
# 1. 答える際にはキーワード[{obs.keyword}] のみを考慮してください。
# 2. はいまたはいいえで答えてください。
# 3. 答えを正確にするために再確認してください。
# 4. 下記のXML形式で回答を提供してください。

# あなたの応答はこの正確な形式であるべきです:
# <answer>yes</answer>
# または
# <answer>no</answer>

# さあ、キーワード[{obs.keyword}]に対して質問に正確に答えてください:
# """
#                           ).run()

In [ ]:
# print(out.prev[-1].content)

In [ ]:
# out.last

# モデルとコードを提出用に圧縮


In [ ]:
!apt install pigz pv

In [ ]:
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py \
    -C /kaggle/working keywords.parquet

In [ ]:
!ls -GFlash --color

# Kaggle CLIを使用して提出

必要に応じて、ノートブックを再実行せずにKaggle CLIインターフェースを使用して提出できます。


In [ ]:
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "ノートブックから提出"